# Dự đoán khả năng sống sót trên tàu Titanic

[Link_to_competition](https://www.kaggle.com/competitions/titanic)

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [2]:
!kaggle competitions download -c titanic


  0%|          | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████| 34.1k/34.1k [00:00<00:00, 485kB/s]


In [3]:
from zipfile import ZipFile
with ZipFile(file='titanic.zip') as f:
    f.extractall(path='titanic_data')

In [4]:
import pandas as pd

In [5]:
raw_df = pd.read_csv('titanic_data/train.csv')
test_df = pd.read_csv('titanic_data/test.csv')
sub_df = pd.read_csv('titanic_data/gender_submission.csv')

In [6]:
raw_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
sub_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [9]:
target_col = 'Survived'
input_cols = raw_df.columns[2:].tolist()

In [10]:
input_cols

['Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [11]:
input_cols.remove('Name')

In [12]:
input_cols.remove('Ticket')

In [13]:
raw_df[input_cols].isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [14]:
import numpy as np

In [15]:
numeric_cols = raw_df[input_cols].select_dtypes(include=np.number).columns.tolist()
categorical_cols = raw_df[input_cols].select_dtypes(include='object').columns.tolist()

In [16]:
numeric_cols.remove('Pclass')
categorical_cols.append('Pclass')

In [17]:
raw_df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [18]:
avg_age = raw_df['Age'].mean()
avg_age

29.69911764705882

In [19]:
raw_df.fillna({'Age': avg_age}, inplace=True)

In [20]:
raw_df[numeric_cols].isna().sum()

Age      0
SibSp    0
Parch    0
Fare     0
dtype: int64

In [21]:
raw_df[input_cols]

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.000000,1,0,7.2500,NaN,S
1,1,female,38.000000,1,0,71.2833,C85,C
2,3,female,26.000000,0,0,7.9250,NaN,S
3,1,female,35.000000,1,0,53.1000,C123,S
4,3,male,35.000000,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.000000,0,0,13.0000,NaN,S
887,1,female,19.000000,0,0,30.0000,B42,S
888,3,female,29.699118,1,2,23.4500,NaN,S
889,1,male,26.000000,0,0,30.0000,C148,C


In [22]:
raw_df['Embarked'] = raw_df['Embarked'].fillna('U')

In [23]:
raw_df['Cabin'].values

array([nan, 'C85', nan, 'C123', nan, nan, 'E46', nan, nan, nan, 'G6',
       'C103', nan, nan, nan, nan, nan, nan, nan, nan, nan, 'D56', nan,
       'A6', nan, nan, nan, 'C23 C25 C27', nan, nan, nan, 'B78', nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, 'D33', nan, 'B30', 'C52', nan, nan, nan,
       nan, nan, 'B28', 'C83', nan, nan, nan, 'F33', nan, nan, nan, nan,
       nan, nan, nan, nan, 'F G73', nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, 'C23 C25 C27', nan, nan, nan, 'E31', nan,
       nan, nan, 'A5', 'D10 D12', nan, nan, nan, nan, 'D26', nan, nan,
       nan, nan, nan, nan, nan, 'C110', nan, nan, nan, nan, nan, nan, nan,
       'B58 B60', nan, nan, nan, nan, 'E101', 'D26', nan, nan, nan,
       'F E69', nan, nan, nan, nan, nan, nan, nan, 'D47', 'C123', nan,
       'B86', nan, nan, nan, nan, nan, nan, nan, nan, 'F2', nan, nan,
       'C2', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [24]:
raw_df['Cabin'] = raw_df['Cabin'].fillna('UNK').copy()

In [25]:
raw_df['Cabin'].value_counts()

Cabin
UNK            687
C23 C25 C27      4
G6               4
B96 B98          4
C22 C26          3
              ... 
E34              1
C7               1
C54              1
E36              1
C148             1
Name: count, Length: 148, dtype: int64

In [26]:
raw_df[input_cols].isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [27]:
inputs = raw_df[input_cols]
targets = raw_df[target_col]

In [28]:
inputs

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.000000,1,0,7.2500,UNK,S
1,1,female,38.000000,1,0,71.2833,C85,C
2,3,female,26.000000,0,0,7.9250,UNK,S
3,1,female,35.000000,1,0,53.1000,C123,S
4,3,male,35.000000,0,0,8.0500,UNK,S
...,...,...,...,...,...,...,...,...
886,2,male,27.000000,0,0,13.0000,UNK,S
887,1,female,19.000000,0,0,30.0000,B42,S
888,3,female,29.699118,1,2,23.4500,UNK,S
889,1,male,26.000000,0,0,30.0000,C148,C


In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
scaler = MinMaxScaler().fit(inputs[numeric_cols])

In [31]:
inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])

C:\Users\admin\AppData\Local\Temp\ipykernel_7544\2993475422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs[numeric_cols] = scaler.transform(inputs[numeric_cols])


In [32]:
inputs[numeric_cols].describe()

,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000
mean,0.367921,0.065376,0.063599,0.062858
std,0.163383,0.137843,0.134343,0.096995
min,0.000000,0.000000,0.000000,0.000000
25%,0.271174,0.000000,0.000000,0.015440
50%,0.367921,0.000000,0.000000,0.028213
75%,0.434531,0.125000,0.000000,0.060508
max,1.000000,1.000000,1.000000,1.000000


In [33]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [35]:
encoder.fit(inputs[categorical_cols])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [36]:
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
encoded_cols

['Sex_female',
 'Sex_male',
 'Cabin_A10',
 'Cabin_A14',
 'Cabin_A16',
 'Cabin_A19',
 'Cabin_A20',
 'Cabin_A23',
 'Cabin_A24',
 'Cabin_A26',
 'Cabin_A31',
 'Cabin_A32',
 'Cabin_A34',
 'Cabin_A36',
 'Cabin_A5',
 'Cabin_A6',
 'Cabin_A7',
 'Cabin_B101',
 'Cabin_B102',
 'Cabin_B18',
 'Cabin_B19',
 'Cabin_B20',
 'Cabin_B22',
 'Cabin_B28',
 'Cabin_B3',
 'Cabin_B30',
 'Cabin_B35',
 'Cabin_B37',
 'Cabin_B38',
 'Cabin_B39',
 'Cabin_B4',
 'Cabin_B41',
 'Cabin_B42',
 'Cabin_B49',
 'Cabin_B5',
 'Cabin_B50',
 'Cabin_B51 B53 B55',
 'Cabin_B57 B59 B63 B66',
 'Cabin_B58 B60',
 'Cabin_B69',
 'Cabin_B71',
 'Cabin_B73',
 'Cabin_B77',
 'Cabin_B78',
 'Cabin_B79',
 'Cabin_B80',
 'Cabin_B82 B84',
 'Cabin_B86',
 'Cabin_B94',
 'Cabin_B96 B98',
 'Cabin_C101',
 'Cabin_C103',
 'Cabin_C104',
 'Cabin_C106',
 'Cabin_C110',
 'Cabin_C111',
 'Cabin_C118',
 'Cabin_C123',
 'Cabin_C124',
 'Cabin_C125',
 'Cabin_C126',
 'Cabin_C128',
 'Cabin_C148',
 'Cabin_C2',
 'Cabin_C22 C26',
 'Cabin_C23 C25 C27',
 'Cabin_C30',
 'Cabin_C3

In [37]:
encoded_df = pd.DataFrame(data=encoder.transform(inputs[categorical_cols]), columns=encoded_cols)
encoded_df.index = inputs.index
encoded_df

,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,Cabin_A20,Cabin_A23,Cabin_A24,Cabin_A26,...,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U,Pclass_1,Pclass_2,Pclass_3
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
888,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
889,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
inputs = pd.concat([inputs, encoded_df], axis=1).copy()

In [39]:
inputs = inputs[numeric_cols+encoded_cols]
inputs

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U,Pclass_1,Pclass_2,Pclass_3
0,0.271174,0.125,0.000000,0.014151,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.472229,0.125,0.000000,0.139136,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.321438,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.434531,0.125,0.000000,0.103644,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.434531,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.334004,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,0.233476,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
888,0.367921,0.125,0.333333,0.045771,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
889,0.321438,0.000,0.000000,0.058556,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [40]:
from sklearn.model_selection import train_test_split

In [41]:
train_inputs, val_inputs, train_targets, val_targets = train_test_split(inputs, targets,
                                                                        test_size=0.2,
                                                                        random_state=42)

In [42]:
train_inputs.shape

(712, 161)

In [43]:
val_inputs.shape

(179, 161)

In [44]:
from xgboost import XGBClassifier

In [45]:
from sklearn.metrics import accuracy_score

In [46]:
model = XGBClassifier(n_jobs=-1, random_state=42, n_estimators=20, max_depth=5)

In [47]:
model.fit(train_inputs, train_targets)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=20, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [48]:
preds = model.predict(train_inputs)
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,

In [49]:
accuracy_score(train_targets, preds)

0.9044943820224719

In [50]:
val_preds = model.predict(val_inputs)

In [51]:
accuracy_score(val_targets, val_preds)

0.8212290502793296

In [52]:
def test_params(**params):
    model = XGBClassifier(n_jobs=-1, random_state=42, **params)
    model.fit(train_inputs, train_targets)
    train_acc = accuracy_score(train_targets, model.predict(train_inputs))
    val_acc = accuracy_score(val_targets, model.predict(val_inputs))
    print('train_acc: {}, val_acc: {}'.format(train_acc, val_acc))

In [53]:
test_params(n_estimators=20, max_depth=4)

train_acc: 0.8918539325842697, val_acc: 0.8212290502793296


In [54]:
for i in [10, 20, 30, 60, 120, 250]:
    print(i, ": ")
    test_params(n_estimators=i)

10 : 
train_acc: 0.901685393258427, val_acc: 0.8379888268156425
20 : 
train_acc: 0.9185393258426966, val_acc: 0.8212290502793296
30 : 
train_acc: 0.9353932584269663, val_acc: 0.8435754189944135
60 : 
train_acc: 0.9592696629213483, val_acc: 0.8156424581005587
120 : 
train_acc: 0.9747191011235955, val_acc: 0.7932960893854749
250 : 
train_acc: 0.9817415730337079, val_acc: 0.7932960893854749


In [55]:
for i in [2, 4, 6, 9, 12, 16, 21, 30]:
    print(i, ": ")
    test_params(n_estimators=30, max_depth=i)

2 : 
train_acc: 0.8581460674157303, val_acc: 0.8324022346368715
4 : 
train_acc: 0.8932584269662921, val_acc: 0.8156424581005587
6 : 
train_acc: 0.9353932584269663, val_acc: 0.8435754189944135
9 : 
train_acc: 0.9536516853932584, val_acc: 0.8044692737430168
12 : 
train_acc: 0.9620786516853933, val_acc: 0.8100558659217877
16 : 
train_acc: 0.9662921348314607, val_acc: 0.8100558659217877
21 : 
train_acc: 0.9634831460674157, val_acc: 0.8044692737430168
30 : 
train_acc: 0.9634831460674157, val_acc: 0.8044692737430168


In [56]:
for i in [0.001, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]:
    print(i, ": ")
    test_params(learning_rate=i, n_estimators=30, max_depth=6)

0.001 : 
train_acc: 0.6235955056179775, val_acc: 0.5865921787709497
0.01 : 
train_acc: 0.8117977528089888, val_acc: 0.770949720670391
0.02 : 
train_acc: 0.8609550561797753, val_acc: 0.8212290502793296
0.05 : 
train_acc: 0.8792134831460674, val_acc: 0.8324022346368715
0.1 : 
train_acc: 0.8960674157303371, val_acc: 0.8435754189944135
0.2 : 
train_acc: 0.9157303370786517, val_acc: 0.8324022346368715
0.3 : 
train_acc: 0.9353932584269663, val_acc: 0.8435754189944135


In [57]:
for i in [0.1, 0.3, 0.5, 0.7, 0.9]:
    print(i, ": ")
    test_params(learning_rate=0.1, n_estimators=30, max_depth=6, sub_sample=i)

0.1 : 
train_acc: 0.8960674157303371, val_acc: 0.8435754189944135

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)



0.3 : 
train_acc: 0.8960674157303371, val_acc: 0.8435754189944135
0.5 : 


C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)


train_acc: 0.8960674157303371, val_acc: 0.8435754189944135
0.7 : 
train_acc: 0.8960674157303371, val_acc: 0.8435754189944135
0.9 : 
train_acc: 0.8960674157303371, val_acc: 0.8435754189944135


C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)


In [58]:
final_model = XGBClassifier(n_jobs=-1, random_state=42,
                            learning_rate=0.1, n_estimators=30, 
                            max_depth=6, sub_sample=0.9,
                            booster='gbtree')

In [59]:
final_model.fit(train_inputs, train_targets)

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [16:29:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "sub_sample" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=30, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [60]:
final_model.score(train_inputs, train_targets)

0.8960674157303371

In [61]:
final_model.score(val_inputs, val_targets)

0.8435754189944135

In [62]:
test_df[numeric_cols].isna().sum()

Age      86
SibSp     0
Parch     0
Fare      1
dtype: int64

In [63]:
test_df.fillna({'Age': avg_age}, inplace=True)

In [64]:
avg_fare = raw_df['Fare'].mean()

In [65]:
test_df.fillna({'Fare': avg_fare}, inplace=True)

In [66]:
test_df[numeric_cols].isna().sum()

Age      0
SibSp    0
Parch    0
Fare     0
dtype: int64

In [67]:
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])

In [68]:
test_df[numeric_cols].describe()

,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000
mean,0.373644,0.055921,0.065391,0.069524
std,0.158792,0.112095,0.163571,0.108994
min,-0.003141,0.000000,0.000000,0.000000
25%,0.283740,0.000000,0.000000,0.015412
50%,0.367921,0.000000,0.000000,0.028213
75%,0.443956,0.125000,0.000000,0.061484
max,0.949736,1.000000,1.500000,1.000000


In [69]:
test_df[categorical_cols].isna().sum()

Sex           0
Cabin       327
Embarked      0
Pclass        0
dtype: int64

In [70]:
test_df.fillna({'Cabin': 'UNK'}, inplace=True)

In [71]:
test_df[categorical_cols].isna().sum()

Sex         0
Cabin       0
Embarked    0
Pclass      0
dtype: int64

In [72]:
encoded_df = pd.DataFrame(data=encoder.transform(test_df[categorical_cols]), columns=encoded_cols)
encoded_df.index = test_df.index
test_df = pd.concat([test_df, encoded_df], axis=1).copy()

In [73]:
test_inputs = test_df[numeric_cols+encoded_cols]
test_inputs

,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,...,Cabin_G6,Cabin_T,Cabin_UNK,Embarked_C,Embarked_Q,Embarked_S,Embarked_U,Pclass_1,Pclass_2,Pclass_3
0,0.428248,0.000,0.000000,0.015282,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.585323,0.125,0.000000,0.013663,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.773813,0.000,0.000000,0.018909,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.334004,0.000,0.000000,0.016908,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.271174,0.125,0.166667,0.023984,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.367921,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
414,0.484795,0.000,0.000000,0.212559,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
415,0.478512,0.000,0.000000,0.014151,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
416,0.367921,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [74]:
test_preds = final_model.predict(test_inputs)

In [75]:
test_preds

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [76]:
accuracy_score(sub_df['Survived'], test_preds)

0.8827751196172249

In [77]:
sub_df['Survived'] = test_preds

In [78]:
sub_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [79]:
sub_df.to_csv('titanic_data/submission.csv', index=None)

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [81]:
base_rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [82]:
base_rf_model.fit(train_inputs, train_targets)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [83]:
base_rf_model.score(train_inputs, train_targets)

0.9845505617977528

In [84]:
base_rf_model.score(val_inputs, val_targets)

0.7877094972067039

In [85]:
def test_params_rf(**params):
    model = RandomForestClassifier(n_jobs=-1, random_state=42, **params)
    model.fit(train_inputs, train_targets)
    train_acc = model.score(train_inputs, train_targets)
    val_acc = model.score(val_inputs, val_targets)
    return {'train_acc': train_acc, 'val_acc': val_acc}

In [86]:
for i in [10, 30, 60, 100, 200, 400, 600]:
    acc = test_params_rf(n_estimators=i)
    print('{}, train_acc: {}, val_acc: {}'.format(i, acc['train_acc'], acc['val_acc']))

10, train_acc: 0.9606741573033708, val_acc: 0.7821229050279329
30, train_acc: 0.9845505617977528, val_acc: 0.8044692737430168
60, train_acc: 0.9845505617977528, val_acc: 0.7988826815642458
100, train_acc: 0.9845505617977528, val_acc: 0.7877094972067039
200, train_acc: 0.9845505617977528, val_acc: 0.7932960893854749
400, train_acc: 0.9845505617977528, val_acc: 0.7877094972067039
600, train_acc: 0.9845505617977528, val_acc: 0.7877094972067039


In [87]:
for i in [5, 10, 15, 20, 25, 30, 35]:
    acc = test_params_rf(n_estimators=30, max_depth=i)
    print('{}, train_acc: {}, val_acc: {}'.format(i, acc['train_acc'], acc['val_acc']))

5, train_acc: 0.8469101123595506, val_acc: 0.8100558659217877
10, train_acc: 0.898876404494382, val_acc: 0.8044692737430168
15, train_acc: 0.9508426966292135, val_acc: 0.7932960893854749
20, train_acc: 0.9676966292134831, val_acc: 0.8212290502793296
25, train_acc: 0.976123595505618, val_acc: 0.8100558659217877
30, train_acc: 0.9831460674157303, val_acc: 0.8100558659217877
35, train_acc: 0.9845505617977528, val_acc: 0.8044692737430168


In [88]:
for i in [1,2,3,5,7,9]:
    acc = test_params_rf(n_estimators=30, max_depth=25, max_leaf_nodes=2**i)
    print('{}, train_acc: {}, val_acc: {}'.format(i, acc['train_acc'], acc['val_acc']))

1, train_acc: 0.7050561797752809, val_acc: 0.7039106145251397
2, train_acc: 0.8019662921348315, val_acc: 0.770949720670391
3, train_acc: 0.8202247191011236, val_acc: 0.7932960893854749
5, train_acc: 0.8820224719101124, val_acc: 0.8100558659217877
7, train_acc: 0.9564606741573034, val_acc: 0.8044692737430168
9, train_acc: 0.9803370786516854, val_acc: 0.7988826815642458


In [89]:
for i in [4,6,8,12,16,24,32,'log2']:
    acc = test_params_rf(n_estimators=30, max_depth=25, max_leaf_nodes=2**5, max_features=i)
    print('{}, train_acc: {}, val_acc: {}'.format(i, acc['train_acc'], acc['val_acc']))

4, train_acc: 0.8665730337078652, val_acc: 0.7988826815642458
6, train_acc: 0.8693820224719101, val_acc: 0.7932960893854749
8, train_acc: 0.8778089887640449, val_acc: 0.8044692737430168
12, train_acc: 0.8820224719101124, val_acc: 0.8100558659217877
16, train_acc: 0.8792134831460674, val_acc: 0.8100558659217877
24, train_acc: 0.8806179775280899, val_acc: 0.8156424581005587
32, train_acc: 0.8876404494382022, val_acc: 0.8156424581005587
log2, train_acc: 0.875, val_acc: 0.8268156424581006


In [90]:
for i in [0,0.1,0.25,0.5,1,1.5]:
    j=i+1
    acc = test_params_rf(n_estimators=30, max_depth=25, 
                         max_leaf_nodes=2**5, max_features='log2',
                         class_weight={0:1, 1:j})
    print('{}, train_acc: {}, val_acc: {}'.format(i, acc['train_acc'], acc['val_acc']))

0, train_acc: 0.875, val_acc: 0.8268156424581006
0.1, train_acc: 0.8792134831460674, val_acc: 0.7932960893854749
0.25, train_acc: 0.8820224719101124, val_acc: 0.8324022346368715
0.5, train_acc: 0.901685393258427, val_acc: 0.8212290502793296
1, train_acc: 0.8679775280898876, val_acc: 0.8044692737430168
1.5, train_acc: 0.8300561797752809, val_acc: 0.7932960893854749


In [91]:
final_rf_model = RandomForestClassifier(n_jobs=-1, random_state=42,
                                        n_estimators=30, max_depth=25,
                                        max_features='log2', max_leaf_nodes=2**5)

In [92]:
final_rf_model.fit(train_inputs, train_targets)

RandomForestClassifier(max_depth=25, max_features='log2', max_leaf_nodes=32,
                       n_estimators=30, n_jobs=-1, random_state=42)

In [93]:
final_rf_model.score(train_inputs, train_targets)

0.875

In [94]:
final_rf_model.score(val_inputs, val_targets)

0.8268156424581006

In [95]:
test_preds = final_rf_model.predict(test_inputs)

In [96]:
accuracy_score(sub_df['Survived'], test_preds)

0.9258373205741627

In [97]:
test_preds

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [98]:
sub_df['Survived'] = test_preds

In [99]:
sub_df.to_csv('titanic_data/submission2.csv', index=None)